# Denoise Network

## Setup

In [1]:
from common import *
!./setup.sh

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Get Denoise Model (Add)

In [4]:
from denoise_network import *
from get_data import get_data

EPOCHS = 50

seqs_train, seqs_test = get_data()

# get traning data
if os.path.exists('data/denoise_data/denoise_generator.npy') and os.path.exists('data/denoise_data/denoise_generator_val.npy'):
    denoise_generator     = np.load('data/denoise_data/denoise_generator.npy'    )
    denoise_generator_val = np.load('data/denoise_data/denoise_generator_val.npy')

else:
    denoise_generator     = DenoiseHPatches(seqs_train, batch_size=500)
    denoise_generator_val = DenoiseHPatches(seqs_test, batch_size=500)
    np.save('data/denoise_data/denoise_generator.npy'    , denoise_generator    )
    np.save('data/denoise_data/denoise_generator_val.npy', denoise_generator_val)


# get model
shape = (32, 32, 1)
denoise_model = get_denoise_model(shape)

# callbacks
callbacks = [
    keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 3, mode= 'auto'),
    keras.callbacks.ModelCheckpoint('data/denoise_model.weights.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)
]

# optimiser
opt  = opt = keras.optimizers.nadam()

# loss
loss = 'mean_absolute_error'

# train network
denoise_model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
denoise_history = denoise_model.fit_generator(generator=denoise_generator, epochs=EPOCHS, callbacks=callbacks,
                                              verbose=1, validation_data=denoise_generator_val)

  5%|▌         | 4/76 [00:03<00:58,  1.24it/s]


KeyboardInterrupt: 